In [11]:
import sys, os, html, re, json, glob, base64, urllib.request, cv2
import tensorflow as tf
import tensorflowjs as tfjs
import numpy as np
from blend_modes import normal
from slpp import slpp as lua

# Make sure working directory is project root
if os.getcwd().rsplit('\\',1)[1]=="model_training":
    os.chdir( os.getcwd().rsplit('\\',1)[0] )

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# Define functions used throughout the document
def read_png(path):
    image = cv2.imread(path,-1)
    return cv2.cvtColor(image,cv2.COLOR_BGR2BGRA) if len(image.shape) > 2 and image.shape[2] == 3 else image

def read_jpeg(path):
    BGR = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    BGRA = cv2.cvtColor(BGR,cv2.COLOR_BGR2BGRA)
    BGRA[:,:,3] = 255
    return BGRA

def layer( image_a, image_b, ratio=1.):
    return normal(image_a.astype(float), image_b.astype(float), ratio).astype(np.uint8)

def save_model(model, model_name):
    model.save                             (os.path.join("model_training","models",model_name,"model.h5"))
    model.save                             (os.path.join("model_training","models",model_name,"model"))
    model.save_weights                     (os.path.join("model_training","models",model_name,"model_weights_checkpoint","model_weights"))
    tfjs.converters.save_keras_model(model, os.path.join("public","models",model_name))
    
def load_model(model_name):
    return tf.keras.models.load_model(os.path.join("model_training","models",model_name,"model"));

def progressbar(it, prefix="", size=60, file=sys.stdout):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()


In [ ]:
def generate_collection_champion_training_images( champion_image_path ):
    
    champion_id = champion_image_path.rsplit(os.path.sep,1)[1].split('.')[0]
    
    image = read_png(champion_image_path)[0:377,:]

    bg = np.zeros((image.shape[0], image.shape[1], image.shape[2]), np.uint8)
    bg[:] = (22, 13, 1, 255)

    image = layer(bg, image, 0.8)    
    image = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_CONSTANT,value=(30,35,40,255))
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)

    for x in range(0, 13, 2):
        for y in range(0, 13, 2):
            crop = image[0+y:image.shape[0]-10+y, 0+x:image.shape[1]-10+x]              
            resized_image = cv2.resize(crop, (28,28), interpolation = cv2.INTER_AREA)
                    
            destination = os.path.join("model_training","training_data","collection_champions",champion_id,f'x{x}_y{y}.png')

            if not os.path.exists(destination.rsplit(os.path.sep,1)[0]):
                os.makedirs(destination.rsplit(os.path.sep,1)[0])

            cv2.imwrite(destination, resized_image)

In [ ]:
coll_champion = glob.glob(os.path.join("model_training","assets","loading_screen_assets","*.png"))
coll_champion = list(filter(lambda path: path.endswith("000.png"), coll_champion))

for path in coll_champion:
    generate_collection_champion_training_images(path)

In [3]:
def generate_collection_skin_training_images( skin_image_path ):
    
    skin_id = skin_image_path.rsplit(os.path.sep,1)[1].split('.')[0]
    
    image = read_png(skin_image_path)[18:409,14:291]

    bg = np.zeros((image.shape[0], image.shape[1], image.shape[2]), np.uint8)
    bg[:] = (22, 13, 1, 255)

    image = layer(bg, image, 0.8)    
    image = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_CONSTANT,value=(30,35,40,255))
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)

    for x in range(0, 13, 2):
        for y in range(0, 13, 2):
            crop = image[0+y:image.shape[0]-10+y, 0+x:image.shape[1]-10+x]              
            resized_image = cv2.resize(crop, (28,28), interpolation = cv2.INTER_AREA)
                    
            destination = os.path.join("model_training","training_data","collection_skins",skin_id,f'x{x}_y{y}.png')

            if not os.path.exists(destination.rsplit(os.path.sep,1)[0]):
                os.makedirs(destination.rsplit(os.path.sep,1)[0])

            cv2.imwrite(destination, resized_image)

In [5]:
coll_skin = glob.glob(os.path.join("model_training","assets","loading_screen_assets","*.png"))
coll_skin = list(filter(lambda path: not path.endswith("000.png"), coll_skin))

for path in coll_skin:
    generate_collection_skin_training_images(path)

In [8]:
coll_champions_image_paths = glob.glob( os.path.join("model_training","training_data","collection_champions","**","*.png") )

print(coll_champions_image_paths[0].rsplit(os.path.sep,2)[1])

1000


In [9]:
# Load up our lookup_champions used for labels
if not os.path.isfile(os.path.join("public","lookup_data","collection_champions.json")):
    exit("Lookup data file (collection_champions.json) from step 2 not found. Exiting.")
with open(os.path.join("public","lookup_data","collection_champions.json")) as json_file:
    lookup_champions  = json.load(json_file)

In [11]:
print(lookup_champions.index("84000"))

2


In [21]:
# Load up our lookup_coll_skins used for labels
if not os.path.isfile(os.path.join("public","lookup_data","collection_skins.json")):
    exit("Lookup data file (collection_skins.json) from step 2 not found. Exiting.")
with open(os.path.join("public","lookup_data","collection_skins.json")) as json_file:
    lookup_coll_skins  = json.load(json_file)

print(lookup_coll_skins[56])

['12020', 1]


In [27]:
print(len(lookup_coll_skins))

1548


In [29]:
model_coll_skins = tf.keras.models.load_model(os.path.join("model_training","models","coll_skins","model"))


In [1]:
os.path.isdir(os.path.join("model_training","models","coll_skins","model"))

False

In [9]:
model_coll_skins = tf.keras.models.load_model(os.path.join("model_training","models","coll_skins","model"))
model_coll_skins.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1548)              1215180   
Total params: 1,215,180
Trainable params: 1,215,180
Non-trainable params: 0
_________________________________________________________________


In [12]:

if not os.path.isfile(os.path.join("public","lookup_data","collection_skins.json")):
    exit("Lookup data file (collection_skins.json) from step 2 not found. Exiting.")

with open(os.path.join("public","lookup_data","collection_skins.json")) as json_file:
    lookup_coll_skins  = json.load(json_file)
    
coll_skins_image_paths = glob.glob( os.path.join("model_training","training_data","collection_skins","**","*.png") )

coll_skins_train_labels = []
coll_skins_train_images = []

for i in progressbar(range(len(coll_skins_image_paths)), "-     Parsing image data: ", 40):

    image = coll_skins_image_paths[i]
    is_limited = "_limited" in image
    skin_id = image.rsplit(os.path.sep,2)[1] if not is_limited else image.rsplit(os.path.sep,2)[1].split("_")[0]
                
    label = lookup_coll_skins.index([skin_id, 1 if is_limited else 0])
    image = read_png(image) / 255

    coll_skins_train_labels.append(label)
    coll_skins_train_images.append(image)

coll_skins_train_images = np.array(coll_skins_train_images)
coll_skins_train_labels = np.array(coll_skins_train_labels)

model_coll_skins = load_model("coll_skins")
probability_model = tf.keras.Sequential([model_coll_skins, tf.keras.layers.Softmax()])
prediction = probability_model.predict(coll_skins_train_images[0])

print(prediction)

# for i in range(len(coll_skins_train_images)):
#     image = coll_skins_train_images[i]

ValueError: in user code:

    C:\Python\Python3.9.5\lib\site-packages\tensorflow\python\keras\engine\training.py:1569 predict_function  *
        return step_function(self, iterator)
    C:\Python\Python3.9.5\lib\site-packages\tensorflow\python\keras\engine\training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Python\Python3.9.5\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Python\Python3.9.5\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Python\Python3.9.5\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Python\Python3.9.5\lib\site-packages\tensorflow\python\keras\engine\training.py:1552 run_step  **
        output